In [1]:
import time
import re
import json
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
# Extract url from settings.json
settings_path = Path("../settings.json")
settings = json.loads(settings_path.open().read())
url = settings["URL1"]

In [3]:
# Initialize the WebDriver
driver = webdriver.Edge()
# driver = webdriver.Chrome()
# driver.execute_script("options.addArguments("--disable-search-engine-choice-screen");").

# Open the homepage
driver.get(url)

# Wait for the page to load
time.sleep(2)

try:
    # Use the id to locate the element
    element = driver.find_element(By.ID, "cookiescript_close")
    element.click()  # Perform the click action
    print("Element clicked successfully!")

except Exception as e:
    print(f"Error occurred while finding/clicking the element: {e}")

The msedgedriver version (128.0.2739.67) detected in PATH at C:\Program Files\webdrivers\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (129.0.2792.52); currently, msedgedriver 129.0.2792.52 is recommended for MicrosoftEdge 129.*, so it is advised to delete the driver in PATH and retry


Element clicked successfully!


In [4]:
try:
    # Use the full class name to locate the element
    element = driver.find_element(By.CLASS_NAME, "MuiTab-iconWrapper")

    # Get the text content of the element
    offers_text = element.text

    # Use regex to extract the integer, allowing for spaces as thousand separators
    offers_number = re.search(
        r"(\d+\s?)+", offers_text
    )  # This captures numbers with optional spaces
    if offers_number:
        # Remove spaces and convert to an integer
        offers_number = int(offers_number.group().replace(" ", ""))
        print("Number of offers:", offers_number)

except Exception as e:
    print(f"Error occurred while finding the element: {e}")

# Wait for results to load
time.sleep(2)

Number of offers: 176


In [5]:
# SCROLL_PAUSE_TIME = 2.0

# # Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

# offset_from_bottom = 0.2 * last_height

# # Scroll down to bottom
# # driver.execute_script(f"window.scrollTo(0, {last_height - offset_from_bottom});")

# while True:
#     # Scroll down to bottom
#     driver.execute_script(f"window.scrollTo(0, {last_height - offset_from_bottom});")

#     # Wait to load page
#     time.sleep(SCROLL_PAUSE_TIME)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

In [7]:
# Locate the parent div (using your unique identifier, e.g., data-test-id)
parent_div = driver.find_element(By.CSS_SELECTOR, '[data-test-id="virtuoso-item-list"]')

# Find all child divs inside the parent div
child_divs = parent_div.find_elements(By.XPATH, "./div")

# Loop through each child div and find nested a tags with links
for index, child_div in enumerate(child_divs):
    # Extract the 'data-index' attribute from each child div
    data_index = child_div.get_attribute("data-index")

    inner_div_1 = child_div.find_element(By.TAG_NAME, "div")
    inner_div_2 = inner_div_1.find_element(By.TAG_NAME, "div")
    # Now locate the <a> tag inside the first inner div
    link_element = inner_div_2.find_element(By.TAG_NAME, "a")
    # Extract the href (link) from the <a> tag
    link = link_element.get_attribute("href")

    position_name = inner_div_2.text.split("\n")[0]
    company_name = inner_div_2.text.split("\n")[3]

    print(data_index)
    print(link)
    print(position_name)
    print(company_name)
    print()

0

1

2

3

4

5

6

7

8

9

10

11

12

13



In [5]:
child_divs

[<selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a1688fe2", element="f.4C16138F27BA34A3899DEFE49B0AA2C0.d.60D43781E7DD67EE732ED3924EFA7060.e.41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a1688fe2", element="f.4C16138F27BA34A3899DEFE49B0AA2C0.d.60D43781E7DD67EE732ED3924EFA7060.e.42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a1688fe2", element="f.4C16138F27BA34A3899DEFE49B0AA2C0.d.60D43781E7DD67EE732ED3924EFA7060.e.43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a1688fe2", element="f.4C16138F27BA34A3899DEFE49B0AA2C0.d.60D43781E7DD67EE732ED3924EFA7060.e.44")>,
 <selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a1688fe2", element="f.4C16138F27BA34A3899DEFE49B0AA2C0.d.60D43781E7DD67EE732ED3924EFA7060.e.45")>,
 <selenium.webdriver.remote.webelement.WebElement (session="16a51138cc07a0011156ee96a

In [71]:
# Close the browser
driver.quit()